<a href="https://colab.research.google.com/github/pksj/IIITA_2nd_SEM/blob/main/MIT2020024_ML_Assignment_01_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Implement K-NN and Logistic regression algorithm from scratch and apply it on MNIST data set.

In [ ]:
import numpy as np
import math

# just to get mnist dataset
from keras.datasets import mnist


#K-NN algorithm

Data Preprocessing

In [ ]:
mnist_dataset = mnist.load_data()


x_train = mnist_dataset[0][0]
y_train = mnist_dataset[0][1]

x_test = mnist_dataset[1][0]
y_test = mnist_dataset[1][1]

#Normalization

x_train = x_train / 255
y_train = y_train / 9

x_test = x_test / 255
y_test = y_test / 9

x_train = x_train[:10000]
y_train = y_train[:10000]

x_test = x_test[:1000]
y_test = y_test[:1000]


Funciton to find distance between two images

In [ ]:
def image_distance(image_1, image_2):

    image_shape = image_1.shape

    distance_square = 0.

    for row in range(image_shape[0]):
        for column in range(image_shape[1]):

            distance_square += (image_1[row][column] - image_2[row][column])**2

    return math.sqrt(distance_square)  



Function max_label

used to find lable with maximum count 

In [ ]:
def max_label(labels):

    frequency = [0] * 10
    
    for label in labels:
        index = int(label * 9 )
        frequency[index] += 1

    max_index = 0

    for i in range(1, 10):

        if(frequency[i] > frequency[max_index]):

            max_index = i

    return max_index / 9   # divid by 9 to make label in normalized range [0/9, 9/9]            


Function KNN_prediction

In [ ]:
def KNN_prediction(k, x_train, y_train, test_image):

    distance_with_labels = []

    for image, label in zip(x_train, y_train):

        ans = image_distance(image, test_image) , label
        #print(ans)
        distance_with_labels.append(ans)

    distance_with_labels.sort(key=lambda x:x[0])

    k_neighbors = [label for image, label in distance_with_labels[0:k]]

    return max_label(k_neighbors)



Prediction on test data

In [ ]:
correct_prediction = 0
c = 0

for image, label in zip(x_test, y_test):

    print(c)
    c += 1

    if(KNN_prediction(10, x_train, y_train, image) == label):


        correct_prediction += 1

print("Accuracy = ", correct_prediction * 100 / len(x_test))        

#Logistic regression algorithm

**Data Preprocessing**

In [ ]:
mnist_dataset = mnist.load_data()


x_train = mnist_dataset[0][0]
y_train = mnist_dataset[0][1]

x_test = mnist_dataset[1][0]
y_test = mnist_dataset[1][1]

x_train = x_train / 255

x_test = x_test / 255

x_train = x_train
y_train = y_train

x_test = x_test
y_test = y_test


y_train_all_digits = np.array([y_train] * 10)
y_test_all_digits = np.array([y_test] * 10)



for i in range(10):
    y_train_all_digits[i] = np.array([ 1 if y == i else 0 for y in y_train_all_digits[i]]) 
    y_test_all_digits[i] = [ 1  if y == i else 0 for y in y_test_all_digits[i]]



In [ ]:
# flattening images

x_train = np.array([np.array(image).ravel() for image in x_train])

x_test = np.array([np.array(image).ravel() for image in x_test])


learning_rate = 0.01
iterations = 100

theta_for_all_digits = [ [0] * len(x_train[0]) for x in range(10)]


**Hypothesis Function**

In [ ]:
def hypothesis(x_data, theta):

    z = np.dot(x_data, theta)
    
    return 1.0  / ( 1.0 +  np.exp(-z))


**Cost Function**

In [ ]:
def cost_function(x_data, y_data, theta):

    cost = 0.0

    for x , y in zip(x_data, y_data):

        h = hypothesis(x,theta)

        cost += (- y) * (np.log(h)) - (1 - y ) * (np.log(1- h))
    
    cost = cost / len(x_data)

    return cost

**Train function**

In [ ]:
def train(iterations, learning_rate, x_data, y_data, theta):

    cost_history = []
    m = len(x_data)
    
    for i in range(iterations):
        
        h = hypothesis(x_data, theta)
        
        dj_by_dw = ( 1 / m ) * np.dot(h - y_data,x_data)
        
        theta = theta - dj_by_dw * learning_rate
        
        cost_history.append(cost_function(x_data, y_data, theta))

    return theta, cost_history

**Determining theta values**

In [ ]:
cost_history_for_all_digits = [0] * 10

for i in range(10):
    theta_for_all_digits[i], cost_history_for_all_digits[i] = train (iterations, learning_rate, x_train, y_train_all_digits[i], theta_for_all_digits[i])


**Predictions**

In [ ]:
correct_prediction = 0;

y_test_all_digits = np.matrix.transpose(y_test_all_digits)

for image, y in zip(x_test, y_test_all_digits):

    j += 1

    probabilities = [0] * 10

    for i in range(10):        
        probabilities[i] = hypothesis(image, theta_for_all_digits[i])

    prediction = probabilities.index(max(probabilities))
    y_argmax = np.argmax(y)
    
    if (prediction == y_argmax):

        correct_prediction += 1


print("accuracy = " , correct_prediction * 100 / len(x_test))
